
---
title: Basic web scraping in Python  
date: 2015-12-23  
comments: false  
tags: Python, web scraping
keywords: python, programming, pandas, matplotlib, web scraping, movielens, christmas, sql  

---



The first step as always with Python analyses is to set up a virtualenv. If you're unfamiliar with how to do this, [this blog post]({filename}2015-11-18-reddit-api-part-1.md) explains how (and I promise you will never want to go back to system-installing packages!). Once you're in your virtualenv, install the following packages:

In [ ]:
!pip install lxml
!pip install cssselect
!pip install requests
!pip install jupyter

We then enter `!ipython notebook` into the command line and start a new notebook (alternatively you can use your preferred Python IDE). 

## 

Once we're ready to go, we first import our newly imported packages.

In [3]:
import lxml.html
from lxml.cssselect import CSSSelector
import requests

The next thing we need to do is find out where our titles are in the page. This is pretty straightforward in a browser that supports developer tools, like [Chrome](https://www.google.com/chrome/). With Chrome, we simply need to go to our list of top 50 movies, right click, and select "Inspect". This brings up the developer tools.

<img src="/figure/web_scraping_inspect_element.png" title="Inspect element button" alt="Finding elements on page in Chrome." style="display: block; margin: auto;" />

As you can see in the image above, this opens up the developer tools on the right of the screen. In the image, I have also highlighted a button that allows you to specifically inspect an element of the page. If you click on this and select one of the movie titles, it will take you to the specific part of the page that contains the title, like so:

<img src="/figure/web_scraping_title_element.png" title="Inspect title" alt="Finding 'The Santa Clause' title in the page." style="display: block; margin: auto;" />

Ah ha! We can see that the first title, 'The Santa Clause', is tagged as `div.feature-item__text h3 a`. However, looking through the rest of the movies (for example, `Joyeux Noël`) are tagged only as `div.feature-item__text h3`. Huh, that creates some problems. To get around this, the function below checks whether the title is tagged with `a` (the `anchor_elem` variable), and if so, it uses this as the title. Otherwise, it uses the `h3` tag at the title.

In [4]:
def get_title(node):
    '''
    Extracts the movie title from the URL http://www.timeout.com/london/film/the-50-best-christmas-movies
    taking into account that some titles are tagged as h3, and some as h3 a.
    '''
    h3_elem = node.cssselect('div.feature-item__text h3')[0]
    anchor_elem = h3_elem.cssselect('a')
    if len(anchor_elem) == 0:
        return h3_elem.text_content()
    else:
        return anchor_elem[0].text_content()

Now that we've set up where to look for the titles, we can extract the data from the website. The `requests.get()` function pulls the data from the website, and the `lxml.html.fromstring(r.text)` command parses the html into the `tree` variable.

In [8]:
# Get data and transform to text
r = requests.get("http://www.timeout.com/london/film/the-50-best-christmas-movies")
tree = lxml.html.fromstring(r.text)

We can now select the parts of the html we want. We can see in the screenshot above that the titles are contained within the `article.feature-item` tag, therefore we select all data under this tag.

In [9]:
items_selector = CSSSelector('article.feature-item')
all_items = items_selector(tree)

We can now apply our `get_title` function using list comprehension to the items we pulled out. Let's have a look at what we got:

In [10]:
h3_titles = [get_title(item) for item in all_items[0:50]]
h3_titles

['The Santa Clause (1994)',
 'Reindeer Games (2000)',
 'The Family Stone (2005)',
 'Love Actually (2003)',
 'Merry Christmas Mr Lawrence (1983)',
 u'\n                                            Joyeux No\xebl (2005)\n                                        ',
 '\n                                            Christmas in Connecticut (1945)\n                                        ',
 'The Polar Express (2004)',
 'A Christmas Story (1983)',
 'The Holiday (2006)',
 'Planes, Trains and Automobiles (1987)',
 'Lethal Weapon (1987)',
 'Ghostbusters II (1989)',
 '\n                                            Prancer (1989)\n                                        ',
 'Holiday Inn (1942)',
 'White Christmas (1954)',
 u'\n                                            Mickey\u2019s Christmas Carol (1983)\n                                        ',
 u'National Lampoon\u2019s Christmas Vacation (1989)',
 '\n                                            Babes In Toyland (1934)\n                         

Ok, this is a bit of a mess. To use it we need to clean it up using a bit of string manipulation.

In [ ]:
# Strip newline and whitespace from titles
titles = [t.replace('\n', '').strip() for t in h3_titles]

In [ ]:
# Convert from unicode and replace apostraphes
titles = [t.encode('utf8').replace('\xe2\x80\x99', '\'') for t in titles]

In [ ]:
# Replace titles in the form "The [title]" to "[title], The"
import re
for i, t in enumerate(titles):
    if re.match("^The", t):
        t = re.sub(r'^The ', '', t)
        titles[i] = t[:-7] + ", The" + t[-7:]
 
# Replace titles in the form "A [title]" to "[title], A"       
for i, t in enumerate(titles):
    if re.match("^A", t):
        t = re.sub(r'^A ', '', t)
        titles[i] = t[:-7] + ", A" + t[-7:]

In [ ]:
# Change "Joyeux Noël" to just "Joyeux" due to special character matching issues        
titles[5] = titles[5].replace('Joyeux No\xc3\xabl (2005)', 
                              'Joyeux')

In [ ]:
# Export to text file                              
f = open("christmas_movies.txt", "w")
f.write("\n".join(map(lambda x: str(x), titles)))
f.close()